## Notebook description/aim

### mapping OmegaA, pco2, pH, using formulations given in MOCSY_sanity_check

In [72]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [73]:
import sys
sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
import mocsy
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
from oct2py import octave
import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn')
import netCDF4 as nc
import numpy as np
import cmocean as cm

""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

In [74]:
reload(VP)

<module 'VIZ_plots' from '/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ/VIZ_plots.py'>

## load dataset and reshape 

In [75]:
w = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_ptrc_T.nc')
wp = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_grid_T.nc')


tDIC = w.variables['dissolved_inorganic_carbon'][0,0,:,:] * 1.e-6
tTA = w.variables['total_alkalinity'][0,0,:,:] * 1.e-6
tTEMP = wp.variables['votemper'][0,0,:,:]
tSAL = wp.variables['vosaline'][0,0,:,:]
tZERO = np.zeros_like(tTEMP)

trDIC = np.ravel(tDIC) 
trTA = np.ravel(tTA)
trTEMP = np.ravel(tTEMP)
trSAL = np.ravel(tSAL)
trZERO = np.ravel(tZERO)
trPATM = np.zeros_like(trZERO)


In [76]:
start = time.time()
response_tup = mocsy.mvars(temp=trTEMP, sal=trSAL, alk=trTA, dic=trDIC, 
                           sil=trZERO, phos=trZERO, patm=trPATM, depth=trZERO, lat=trZERO, 
                            optcon='mol/kg', optt='Tpot', optp='m',
                            optb='l10', optk1k2='m10',  optkf='pf', optgas='Ppot')
pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup


"the code you want to test stays here"
end = time.time()
print(end - start)

0.5583744049072266


In [77]:
#task: reshape vars back 

tpH = pH.reshape(898,398)
tpco2 = pco2.reshape(898,398)
tfco2 = fco2.reshape(898,398)
tco2 = co2.reshape(898,398)
thco3 = hco3.reshape(898,398)
tco3 = co3.reshape(898,398)
tOmegaA = OmegaA.reshape(898,398)
tOmegaC = OmegaC.reshape(898,398)

In [78]:
# VP.phpco2om_plot \
#     (tpH,tpco2,tOmegaA,'pH','pCO2','OmegaA',cm.cm.speed,cm.cm.matter, \
#      cm.cm.balance,18,11,7.6,8.2,0,800,0,2,'pH','pCO2','Omega A','Carbonate Chemistry Parameters')

In [79]:
w = nc.Dataset('/data/tjarniko/results/may10_a7/SKOG_1d_20160331_20160414_ptrc_T.nc')
wp = nc.Dataset('/data/tjarniko/results/may10_a7/SKOG_1d_20160331_20160414_grid_T.nc')

for i in range(0,15):
    day = i
    tDIC = w.variables['dissolved_inorganic_carbon'][i,0,:,:] * 1.e-6
    tTA = w.variables['total_alkalinity'][i,0,:,:] * 1.e-6
    tTEMP = wp.variables['votemper'][i,0,:,:]
    tSAL = wp.variables['vosaline'][i,0,:,:]
    tZERO = np.zeros_like(tTEMP)

    trDIC = np.ravel(tDIC) 
    trTA = np.ravel(tTA)
    trTEMP = np.ravel(tTEMP)
    trSAL = np.ravel(tSAL)
    trZERO = np.ravel(tZERO)
    trPATM = np.zeros_like(trZERO)

    response_tup = mocsy.mvars(temp=trTEMP, sal=trSAL, alk=trTA, dic=trDIC, 
                               sil=trZERO, phos=trZERO, patm=trPATM, depth=trZERO, lat=trZERO, 
                                optcon='mol/kg', optt='Tinsitu', optp='db',
                                optb = 'u74', optk1k2='l', optkf = 'pf', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    tpH = pH.reshape(898,398)
    tpco2 = pco2.reshape(898,398)
    tfco2 = fco2.reshape(898,398)
    tco2 = co2.reshape(898,398)
    thco3 = hco3.reshape(898,398)
    tco3 = co3.reshape(898,398)
    tOmegaA = OmegaA.reshape(898,398)
    tOmegaC = OmegaC.reshape(898,398)
    tOmegaA = np.ma.masked_values(tOmegaA,1e+20)
    tpH = np.ma.masked_values(tpH,1e+20)
    tpco2 = np.ma.masked_values(tpco2,1e+20)
    ind = i
    if i < 10:
        ind = '0' + str(i)
    tit = 'Carbonate Chemistry Parameters, Day ' + str(ind)
    
    VP.phpco2om_plot \
        (tpH,tpco2,tOmegaA,'pH','pCO2','OmegaA',cm.cm.speed,cm.cm.matter, \
         cm.cm.balance,18,11,7.6,8.2,0,800,0,2,'pH','pCO2','Omega A',tit,'./',ind)

#     dirstr = './'
#     figtit = 'CCpar_day_' + str(i)
#     total_fig = dirstr+figtit + '.png'
#     fig.savefig(total_fig)

    
    print(ind)

00
01
02
03
04
05
06
07
08
09
10
11
12
13
14


In [80]:
w = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_ptrc_T.nc')
wp = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_grid_T.nc')


for i in range(0,15):
    day = i
    tDIC = w.variables['dissolved_inorganic_carbon'][i,0,:,:] * 1.e-6
    tTA = w.variables['total_alkalinity'][i,0,:,:] * 1.e-6
    tTEMP = wp.variables['votemper'][i,0,:,:]
    tSAL = wp.variables['vosaline'][i,0,:,:]
    tZERO = np.zeros_like(tTEMP)

    trDIC = np.ravel(tDIC) 
    trTA = np.ravel(tTA)
    trTEMP = np.ravel(tTEMP)
    trSAL = np.ravel(tSAL)
    trZERO = np.ravel(tZERO)
    trPATM = np.zeros_like(trZERO)

    response_tup = mocsy.mvars(temp=trTEMP, sal=trSAL, alk=trTA, dic=trDIC, 
                               sil=trZERO, phos=trZERO, patm=trPATM, depth=trZERO, lat=trZERO, 
                                optcon='mol/kg', optt='Tinsitu', optp='db',
                                optb = 'u74', optk1k2='l', optkf = 'pf', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    tpH = pH.reshape(898,398)
    tpco2 = pco2.reshape(898,398)
    tfco2 = fco2.reshape(898,398)
    tco2 = co2.reshape(898,398)
    thco3 = hco3.reshape(898,398)
    tco3 = co3.reshape(898,398)
    tOmegaA = OmegaA.reshape(898,398)
    tOmegaC = OmegaC.reshape(898,398)
    tOmegaA = np.ma.masked_values(tOmegaA,1e+20)
    tpH = np.ma.masked_values(tpH,1e+20)
    tpco2 = np.ma.masked_values(tpco2,1e+20)

    ind = i+15
    
    tit = 'Carbonate Chemistry Parameters, Day ' + str(ind)
    
    VP.phpco2om_plot \
        (tpH,tpco2,tOmegaA,'pH','pCO2','OmegaA',cm.cm.speed,cm.cm.matter, \
         cm.cm.balance,18,11,7.6,8.2,0,800,0,2,'pH','pCO2','Omega A',tit,'./',ind)

#     dirstr = './'
#     figtit = 'CCpar_day_' + str(i)
#     total_fig = dirstr+figtit + '.png'
#     fig.savefig(total_fig)
    #plt.show()
    
    print(ind)

15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [81]:
HTML("""
<video width="960" height="720" controls>
  <source src="./movie_CCpar_day_.mp4" type="video/mp4">
</video>
""")

NameError: name 'nc' is not defined